In [6]:
# Env
import gymnasium as gym
from torchrl.envs import GymEnv, ParallelEnv, TransformedEnv, Compose, DTypeCastTransform, BatchSizeTransform
from env import ChessEnv

gym.register(
    id='Chess-v0',
    entry_point='env:ChessEnv',
)

env = ParallelEnv(
    num_workers=4,
    create_env_fn=lambda: GymEnv(
        "Chess-v0",
        categorical_action_encoding=True,
        render_mode="human",
    ),
)

n_obs = env.observation_spec["observation"].shape[-1]
n_actions = env.action_spec.n

/home/bryan/github/cpp/CS4200/chess/.venv/lib/python3.12/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment Chess-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [2]:
# Policy
from tensordict.nn import TensorDictModule
from torch.distributions import Categorical
from torchrl.modules import MLP, ValueOperator, ProbabilisticActor
from actor import ChessActor

actor_mod = TensorDictModule(
    module=ChessActor(n_obs, n_actions),
    in_keys=["observation"],
    out_keys=["logits"],
)

actor = ProbabilisticActor(
    module=actor_mod,
    in_keys=["logits"],
    out_keys=["action"],
    distribution_class=Categorical,
    spec=env.action_spec,
)

critic_net = MLP(in_features=n_obs, out_features=1)

critic = ValueOperator(
    module=critic_net,
    in_keys=["observation"],
    out_keys=["state_value"],
)

In [3]:
# Optimizer
from torchrl.objectives import PPOLoss
from torchrl.objectives.value import GAE

loss_fn = PPOLoss(
    actor_network=actor,
    critic_network=critic,
    action_spec=env.action_spec,
    value_loss_coef=0.5,
    entropy_coef=0.01,
)

advantage = GAE(
    value_network=critic,
    gamma=0.99,
    lmbda=0.95,
)

In [7]:
# Training
for i in range(1):
    rollout = env.rollout(
        policy=actor,
        max_steps=1,
    )

/home/bryan/github/cpp/CS4200/chess/.venv/lib/python3.12/site-packages/torchrl/data/replay_buffers/samplers.py:37: UserWarning: Failed to import torchrl C++ binaries. Some modules (eg, prioritized replay buffers) may not work with your installation. This is likely due to a discrepancy between your package version and the PyTorch version. Make sure both are compatible. Usually, torchrl majors follow the pytorch majors within a few days around the release. For instance, TorchRL 0.5 requires PyTorch 2.4.0, and TorchRL 0.6 requires PyTorch 2.5.0.
  warnings.warn(EXTENSION_WARNING)
/home/bryan/github/cpp/CS4200/chess/.venv/lib/python3.12/site-packages/torchrl/data/replay_buffers/samplers.py:37: UserWarning: Failed to import torchrl C++ binaries. Some modules (eg, prioritized replay buffers) may not work with your installation. This is likely due to a discrepancy between your package version and the PyTorch version. Make sure both are compatible. Usually, torchrl majors follow the pytorch ma

EOFError: 